# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 10 2023 8:43AM,254628,20,9013816,"Exact-Rx, Inc.",Released
1,Jan 10 2023 8:36AM,254626,10,9013839,Bio-PRF,Released
2,Jan 10 2023 8:32AM,254625,20,9013838,"Andelyn Biosciences, Inc.",Released
3,Jan 10 2023 8:30AM,254615,10,0086286484,ISDIN Corporation,Released
4,Jan 10 2023 8:30AM,254615,10,0086286488,ISDIN Corporation,Released
5,Jan 10 2023 8:30AM,254615,10,0086286495,ISDIN Corporation,Released
6,Jan 10 2023 8:30AM,254615,10,0086286497,ISDIN Corporation,Released
7,Jan 10 2023 8:30AM,254615,10,0086286509,ISDIN Corporation,Released
8,Jan 10 2023 8:30AM,254615,10,0086286510,ISDIN Corporation,Released
9,Jan 10 2023 8:30AM,254615,10,0086286512,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,254622,Released,1
27,254623,Released,42
28,254625,Released,1
29,254626,Released,1
30,254628,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254622,NaN,NaN,1.0
254623,NaN,NaN,42.0
254625,NaN,NaN,1.0
254626,NaN,NaN,1.0
254628,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254533,2.0,0.0,1.0
254558,0.0,1.0,0.0
254562,0.0,1.0,0.0
254563,37.0,2.0,1.0
254566,0.0,0.0,36.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254533,2,0,1
254558,0,1,0
254562,0,1,0
254563,37,2,1
254566,0,0,36


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254533,2,0,1
1,254558,0,1,0
2,254562,0,1,0
3,254563,37,2,1
4,254566,0,0,36


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254533,2,,1
1,254558,,1,
2,254562,,1,
3,254563,37,2,1
4,254566,,,36


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 10 2023 8:43AM,254628,20,"Exact-Rx, Inc."
1,Jan 10 2023 8:36AM,254626,10,Bio-PRF
2,Jan 10 2023 8:32AM,254625,20,"Andelyn Biosciences, Inc."
3,Jan 10 2023 8:30AM,254615,10,ISDIN Corporation
10,Jan 10 2023 7:46AM,254623,19,"AdvaGen Pharma, Ltd"
52,Jan 10 2023 7:40AM,254622,22,"NBTY Global, Inc."
53,Jan 9 2023 4:55PM,254617,15,"Person & Covey, Inc."
56,Jan 9 2023 4:27PM,254616,15,Lonza
57,Jan 9 2023 4:04PM,254613,15,"Person & Covey, Inc."
63,Jan 9 2023 4:00PM,254612,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 10 2023 8:43AM,254628,20,"Exact-Rx, Inc.",,,1
1,Jan 10 2023 8:36AM,254626,10,Bio-PRF,,,1
2,Jan 10 2023 8:32AM,254625,20,"Andelyn Biosciences, Inc.",,,1
3,Jan 10 2023 8:30AM,254615,10,ISDIN Corporation,,,7
4,Jan 10 2023 7:46AM,254623,19,"AdvaGen Pharma, Ltd",,,42
5,Jan 10 2023 7:40AM,254622,22,"NBTY Global, Inc.",,,1
6,Jan 9 2023 4:55PM,254617,15,"Person & Covey, Inc.",,,3
7,Jan 9 2023 4:27PM,254616,15,Lonza,,,1
8,Jan 9 2023 4:04PM,254613,15,"Person & Covey, Inc.",,,6
9,Jan 9 2023 4:00PM,254612,19,"GUSA Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 8:43AM,254628,20,"Exact-Rx, Inc.",1,,
1,Jan 10 2023 8:36AM,254626,10,Bio-PRF,1,,
2,Jan 10 2023 8:32AM,254625,20,"Andelyn Biosciences, Inc.",1,,
3,Jan 10 2023 8:30AM,254615,10,ISDIN Corporation,7,,
4,Jan 10 2023 7:46AM,254623,19,"AdvaGen Pharma, Ltd",42,,
5,Jan 10 2023 7:40AM,254622,22,"NBTY Global, Inc.",1,,
6,Jan 9 2023 4:55PM,254617,15,"Person & Covey, Inc.",3,,
7,Jan 9 2023 4:27PM,254616,15,Lonza,1,,
8,Jan 9 2023 4:04PM,254613,15,"Person & Covey, Inc.",6,,
9,Jan 9 2023 4:00PM,254612,19,"GUSA Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 8:43AM,254628,20,"Exact-Rx, Inc.",1,,
1,Jan 10 2023 8:36AM,254626,10,Bio-PRF,1,,
2,Jan 10 2023 8:32AM,254625,20,"Andelyn Biosciences, Inc.",1,,
3,Jan 10 2023 8:30AM,254615,10,ISDIN Corporation,7,,
4,Jan 10 2023 7:46AM,254623,19,"AdvaGen Pharma, Ltd",42,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 8:43AM,254628,20,"Exact-Rx, Inc.",1.0,NaN,NaN
1,Jan 10 2023 8:36AM,254626,10,Bio-PRF,1.0,NaN,NaN
2,Jan 10 2023 8:32AM,254625,20,"Andelyn Biosciences, Inc.",1.0,NaN,NaN
3,Jan 10 2023 8:30AM,254615,10,ISDIN Corporation,7.0,NaN,NaN
4,Jan 10 2023 7:46AM,254623,19,"AdvaGen Pharma, Ltd",42.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 8:43AM,254628,20,"Exact-Rx, Inc.",1.0,0.0,0.0
1,Jan 10 2023 8:36AM,254626,10,Bio-PRF,1.0,0.0,0.0
2,Jan 10 2023 8:32AM,254625,20,"Andelyn Biosciences, Inc.",1.0,0.0,0.0
3,Jan 10 2023 8:30AM,254615,10,ISDIN Corporation,7.0,0.0,0.0
4,Jan 10 2023 7:46AM,254623,19,"AdvaGen Pharma, Ltd",42.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1018354,10.0,0.0,5.0
15,2036800,81.0,2.0,37.0
16,254585,1.0,0.0,0.0
19,1527581,56.0,0.0,0.0
20,1018373,2.0,2.0,0.0
21,763794,3.0,0.0,0.0
22,254622,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1018354,10.0,0.0,5.0
1,15,2036800,81.0,2.0,37.0
2,16,254585,1.0,0.0,0.0
3,19,1527581,56.0,0.0,0.0
4,20,1018373,2.0,2.0,0.0
5,21,763794,3.0,0.0,0.0
6,22,254622,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,10.0,0.0,5.0
1,15,81.0,2.0,37.0
2,16,1.0,0.0,0.0
3,19,56.0,0.0,0.0
4,20,2.0,2.0,0.0
5,21,3.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,10.0
1,15,Released,81.0
2,16,Released,1.0
3,19,Released,56.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,5.0,37.0,0.0,0.0,0.0,0.0,0.0
Executing,0.0,2.0,0.0,0.0,2.0,0.0,0.0
Released,10.0,81.0,1.0,56.0,2.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,5.0,37.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,2.0,0.0,0.0,2.0,0.0,0.0
2,Released,10.0,81.0,1.0,56.0,2.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,5.0,37.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,2.0,0.0,0.0,2.0,0.0,0.0
2,Released,10.0,81.0,1.0,56.0,2.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()